### Librerías

In [ ]:
import geopandas as gpd
import pandas as pd
from Valor_catastral_construccion import *
from Valor_catastral_terreno import *
from calculo_predial import *
import random
import numpy as np 


### Base de datos
Todo este procedimiento a implementar es un seguimiento detallado del [manual catastral](!https://igecem.edomex.gob.mx/sites/igecem.edomex.gob.mx/files/files/ArchivosPDF/Servicios-catastrales/Manual%20Catastral%20del%20Estado%20de%20Mexico.pdf) que el IGECEM publica para el calculo del valor catastral. 

In [ ]:
path_hector = r"/home/hector/Documentos/Infis/Geo/Data/NuevoModelo_Naucalpan_pre_valor_cat.csv"
#path_ruben = r"G:\.shortcut-targets-by-id\1xy7wVmj8pxJHqeyV9CG1vOBQs3emChdq\Geo\Data\Completados\Modelo_Final\Naucalpan_Final.xlsx"
datos = pd.read_csv(path_hector, encoding='utf8', memory_map=True)
#datos = pd.read_excel(path_ruben,nrows=200)
datos.fillna(0,inplace=True)
datos.shape

In [ ]:
vu_const = pd.read_excel('./Data/Valor de construccion unitario habitacional naucalpan.xlsx')
vu_const.shape

In [ ]:
# Obtenemos las columnas de altura, nivel, y valores para borrarlas, ya que se hara de nuevo la clasificacion
cols_altura = np.array([])
cols_nivel = np.array([])
cols_valor_uni = np.array([])
cols_otros = np.array([])
col_categoria = None
datos.fillna(0, inplace=True)


for col in datos:
    if col.lower().strip().startswith('altura'):
        cols_altura = np.append(cols_altura, col)
    elif col.lower().strip().startswith('nivel'):
        cols_nivel = np.append(cols_nivel, col)
    elif col.lower().strip().startswith('val'):
        cols_valor_uni = np.append(cols_valor_uni,col)
    elif col in list(vu_const['Categoría']):
        cols_otros = np.append(cols_otros, col)
    elif col.lower().strip().startswith('clase_dete'):
        col_categoria = col
len(cols_altura), len(cols_nivel), len(cols_valor_uni), len(cols_otros), col_categoria

In [ ]:
# Revisamos las categorias
datos[col_categoria].value_counts()

In [ ]:
# Borramos la categoria de carros
shape1 = datos.shape[0]
datos.drop(index=datos[datos['clase_dete']=='carros'].index, inplace=True)
'Registros borrados = ',shape1 - datos.shape[0]

In [ ]:
# Columnas de categorias
casas = ['casas', 'establecimiento','multivivienda']
terrenos = ['area_verde','terreno_baldio',]

In [ ]:
datos.loc[datos[col_categoria].isin(casas)]

### Categorizacion por intervalos

In [ ]:
dict_intervalos = {
    'Medio'    :(60,250),
    'Popular'  :(40,80),
    'Interes'  :(0,70),
    'Campestre':(50, 400),
    'Rural'    :(50, 400),
    'Comercial':(400,10000000000000000)
}
dict_niveles = {
    'NivelesM_HMEA' :18,
    'NivelesM_HMEM' :10,
    'NivelesM_HMEB' :4,
    'NivelesP_HPOM' :8,
    'NivelesP_HPOB' :6,
    'NivelesIS_HISM':8,
    'NivelesIS_HISB':6,
    'NivelesC_HCAM' :2,
    'NivelesR_HRUR' :3,
    'NivelesCom'    :14
}
dict_alturas = {
    'AlturaM_HMEA' :75.6,
    'AlturaM_HMEM' :42,
    'AlturaM_HMEB' :16.8,
    'AlturaP_HPOM' :33.6,
    'AlturaP_HPOB' :25.2,
    'AlturaIS_HISM':33.6,
    'AlturaIS_HISB':25.2,
    'AlturaC_HCAM' :8.4,
    'AlturaR_HRUR' :12.6,
    'AlturaCom'    :35
}
dict_construccion = {}
for i in range(len(vu_const)):
  dict_construccion[str(vu_const['Categoría'][i])]=(vu_const['min'][i],vu_const['Max'][i])

In [ ]:
# Categorizar superficies por valor de construccion
import warnings
warnings.simplefilter('ignore')
for key, val in dict_construccion.items():
    print(key, val)
    datos.loc[datos['clase_dete'].isin(casas),key] = datos[datos['clase_dete'].isin(casas)]['Area_Model'].map(lambda x: Categorizar_por_intervalos(x, val[0], val[1]) )

In [ ]:
datos.columns

In [ ]:
# Categorizar superficies
for key, val in dict_intervalos.items():
    print(key, val)
    datos[key] = datos['Area_Model'].map(lambda x: Categorizar_por_intervalos(x, val[0], val[1]) )

In [ ]:
# Poner niveles maximos
for key,val in dict_niveles.items():
    print(key,val)
    datos[key] = val
    if key.find('AlturaM')>=0:
        datos[key] = datos[key]*datos['Medio']
    elif key.find('AlturaP') >= 0:
        datos[key] = datos[key]*datos['Popular']
    elif key.find('AlturaIS') >= 0:
        datos[key] = datos[key]*datos['Interes']
    elif key.find('AlturaC') >= 0:
        datos[key] = datos[key]*datos['Campestre']
    elif key.find('AlturaR') >= 0:
        datos[key] = datos[key]*datos['Rural']
    elif key.find('AlturaCom') >= 0:
        datos[key] = datos[key]*datos['Comercial']
datos['Nivel_minimo'] = 1

In [ ]:
# Poner alturas maximas
for key,val in dict_alturas.items():
    print(key,val)
    datos[key] = val
    if key.find('NivelesM')>=0:
        datos[key] = datos[key]*datos['Medio']
    elif key.find('NivelesP') >= 0:
        datos[key] = datos[key]*datos['Popular']
    elif key.find('NivelesIS') >= 0:
        datos[key] = datos[key]*datos['Interes']
    elif key.find('NivelesC') >= 0:
        datos[key] = datos[key]*datos['Campestre']
    elif key.find('NivelesR') >= 0:
        datos[key] = datos[key]*datos['Rural']
    elif key.find('NivelesCom') >= 0:
        datos[key] = datos[key]*datos['Comercial']
datos['Altura_min'] = 2.5

In [ ]:
datos.columns

### Llenado de datos faltantes

In [ ]:
# Obtenemos las columnas de altura, nivel, y valores
cols_altura = np.array([])
cols_nivel = np.array([])
cols_valor_uni = np.array([])
cols_const = np.array([])
datos.fillna(0, inplace=True)

for col in datos:
    if col.lower().strip().startswith('altura'):
        cols_altura = np.append(cols_altura, col)
    elif col.lower().strip().startswith('nivel'):
        cols_nivel = np.append(cols_nivel, col)
    elif col.lower().strip().startswith('val'):
        cols_valor_uni = np.append(cols_valor_uni,col)
    elif col in list(vu_const['Categoría']):
        cols_const = np.append(cols_const,col)
len(cols_altura), len(cols_nivel), len(cols_valor_uni), len(cols_const)

In [ ]:
datos['Frente_base'] = datos['Frente']
datos['Fondo_base']  = datos['Fondo']
datos['Area_base']   = datos['Area_AH']

datos.loc[datos[col_categoria].isin(casas),'grado_conservacion'] = datos.loc[datos[col_categoria].isin(casas)]['Area_AH'].map(lambda x:get_grado_conservacion())
datos['Area_inscrita']     = datos['Area_Model'].map(lambda x: float(x)*random.uniform(0.65,1))
datos['Area_construccion'] = datos['Area_Model']*.9

### Calculo de factores

In [ ]:
cols_factor_top = np.array([])
cols_factor_nvl = np.array([])

In [ ]:
# Factores catastral del terreno
datos.fillna(0, inplace=True)
datos['Factor_posicion']      = datos['Esquinero|'].fillna(0).map(lambda x: get_factor_posicion(x))
datos['Terreno_Posicion']     = datos['Factor_posicion'].map(lambda x: 'Interior' if x==0.5 else 'otro')
#datos['factor_frente']        = pd.Series(map(factor_frente,datos['Medida_lx'].fillna(0),datos['Terreno_Posicion'] ))
datos['factor_frente']        = datos.apply(lambda x: factor_frente(x['Medida_lx'], x['Terreno_Posicion']),axis=1)
#datos['factor_fondo']         = pd.Series(map(factor_fondo,datos['Medida_ly'].fillna(0), datos['Fondo_base'].fillna(0),datos['Terreno_Posicion']))
datos['factor_fondo']         = datos.apply(lambda x: factor_fondo(x['Medida_ly'], x['Fondo_base'], x['Terreno_Posicion']), axis=1)
for col in cols_altura:
    nombre_col = 'Factor_Topografia_'+col
    cols_factor_top = np.append(cols_factor_top, nombre_col)
    datos[nombre_col] = datos.apply(lambda x: factor_topografia(x[col], x['Medida_ly'], x['Terreno_Posicion']), axis=1)

#datos['factor_irregularidad'] = pd.Series(map(factor_irregularidad,datos['Area_Model'], datos['Area_inscrita'], datos['Terreno_Posicion']))
datos['factor_irregularidad'] = datos.apply(lambda x: factor_irregularidad(x['Area_Model'], x['Area_inscrita'], x['Terreno_Posicion']), axis=1)
#datos['factor_area']          = pd.Series(map(factor_area,datos['Area_Model'], datos['Area_base'],datos['Terreno_Posicion']))
datos['factor_area']          = datos.apply(lambda x: factor_area(x['Area_Model'],x['Area_base'],x['Terreno_Posicion']), axis=1)
#datos['factor_restriccion']   = pd.Series(map(factor_restriccion,datos['Area_Model'], datos['Area_Model'],datos['Terreno_Posicion']))
datos['factor_restriccion']   = datos.apply(lambda x: factor_restriccion(x['Area_Model'],x['Area_Model']*0.9,x['Terreno_Posicion']), axis=1)

In [ ]:
# Revison de valores 0 en los factores de terreno y correccion
cols_ft = {'factor_frente':0.5,'factor_fondo':0.6,'factor_irregularidad':0.5,'factor_area':0.7,'factor_restriccion':0.5}

for col, val in cols_ft.items():
    s = datos[datos[col].fillna(0)==0].shape
    
    if s[0] != 0:
        datos.loc[datos[col].fillna(0)==0, col] = val
        print(col)
    print(s)
    
for col in cols_factor_top:
    s = datos[datos[col].fillna(0)==0].shape
    if s[0] != 0:
        print(col)
    print(s)
    

In [ ]:
# Factores catastral de la construcción
datos.loc[datos[col_categoria].isin(casas),'factor_grado_conservacion'] = datos[datos[col_categoria].isin(casas)]['grado_conservacion'].map(lambda x: factor_grado_conservacion(x))
datos.loc[datos[col_categoria].isin(casas),'factor_edad']               = 0.5

for col in cols_nivel:
    nombre = 'factor_numero_niveles_'+col
    cols_factor_nvl = np.append(cols_factor_nvl, nombre)
    datos.loc[datos[col_categoria].isin(casas),nombre] = datos[datos[col_categoria].isin(casas)].apply(lambda x: factor_numero_niveles(x[col],x['grado_conservacion']), axis=1)

In [ ]:
# Revision de valores 0 en los factores de construccion
#datos.loc[(datos[col_categoria].isin(casas))&(datos['factor_grado_conservacion'].fillna(0)==0.0),nombre] =
print(datos[(datos[col_categoria].isin(casas))&(datos['factor_grado_conservacion'].fillna(0)==0.0)].shape)
print(datos[(datos[col_categoria].isin(casas))&(datos['factor_edad'].fillna(0)==0.0)].shape)
#print(datos[datos['factor_edad'].fillna(0)==0.0].shape)
for col in cols_factor_nvl:
    s = datos[(datos[col_categoria].isin(casas))&(datos[col].fillna(0)==0.0)].shape
    if s[0] != 0:
        print(col)
    print(s)

### Calculo de valores catastrales

In [ ]:
cols_using = ['Area_Model','factor_frente','factor_fondo', 'factor_irregularidad','factor_area',
              'Factor_posicion','factor_restriccion']
datos.fillna(0,inplace=True)
datos['Val m2_Man'] = datos['Val m2_Man'].astype(str).str.replace(',','')
for col in cols_using:
    datos[col] = datos[col].astype(float)

![alt text](Data\Valor_cat_terreno.png) 

Formula para obtener el valor catastral del terreno

In [ ]:
cols_factor_top

In [ ]:
# Valor catastral del terreno
cols_vc_terr = np.array([])
for col_vu in cols_valor_uni:
    i = 0
    name = col_vu.split('_')[-1]
    for col_ft in cols_factor_top:
        name_col = 'Valor_catastral_terreno_'+name+'_'+col_ft.split('_')[-1]
        datos[name_col] = datos['Area_Model'].astype(float).fillna(0)*datos[col_vu].astype(float).fillna(0)*datos['factor_frente'].astype(float).fillna(0)*datos['factor_fondo'].fillna(0)*datos['factor_irregularidad'].fillna(0)*datos['factor_area'].fillna(0)*datos[col_ft].fillna(0)*datos['Factor_posicion'].fillna(0)*datos['factor_restriccion'].fillna(0)
        i += 1
        cols_vc_terr = np.append(cols_vc_terr,name_col)

![alt text](Data\Valor_cat_const.png) <br>
 Formula para obtener el valor catastral de la construccion

In [ ]:
# Valor catastral de la construccion
cols_vc_const = np.array([])
for col_vu in cols_const:
    i = 0
    name = col_vu.split('_')[-1]
    for col_nv in cols_factor_nvl:
        name_col = 'Valor_catastral_construccion_'+name+'_'+str(i)
        cols_vc_const = np.append(cols_vc_const,name_col)
        datos[name_col] = datos['Area_Model'].astype(float).fillna(0)*datos[col_vu].astype(float).fillna(0)*datos['factor_edad'].astype(float).fillna(0)*datos['factor_grado_conservacion'].astype(float).fillna(0)*datos[col_nv].astype(float).fillna(0)
        i += 1

In [ ]:
len(cols_vc_const), len(cols_vc_terr), datos.shape

In [ ]:
# Revisamos cuantos obtuvieron 0 en todos los valores
pt = datos.apply(lambda x: np.mean(x[cols_vc_terr]), axis=1)
pt.value_counts()

![alt text](Data\Valor_cat_final.png)

Formula para obtener el valor catastral final 

### Exportación de datos

In [ ]:
datos.to_csv(r"C:\Users\ruben\Downloads\Prueba_muestra_100_valor_catastral.csv", 
             index=False, encoding='utf-8')